# batteries

Calculations and estimation for battery data

In [1]:
#| default_exp battery

In [2]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *
from matplotlib import pyplot as plt
import pandas as pd

In [3]:
#| export
from triumph.motor import evaluate
from triumph.utilities import *
import numpy as np
from fastcore.utils import *

## Defining a battery

We're focusing on using `prismatic` cells, which are the most common type of cell used in electric vehicles. These cells are rectangular in shape, and are typically packaged into a module with other cells. The module is then packaged into a pack with other modules. The pack is the unit that is installed into the vehicle.

Typical voltages of a single prismatic cell are 3.2V, 3.6V, and 3.8V. The capacity of a cell is typically between 1 and 10 Ah. The capacity of a module is typically between 10 and 100 Ah. The capacity of a pack is typically between 100 and 1000 Ah.

In [4]:
#| export

class Cell():
  """ Defines a single cell. """
  def __init__(self,
               name='Generic Cell',
               voltage=3.2, # V DC, nominal
               capacity=140, # Ah
               dimensions = {'length':None, 'width':None, 'height':None}, # inches
               mass = None, # lbs
               cost = None, # USD
               ):
    self.voltage = voltage
    self.capacity = capacity
    self.kWh = capacity*voltage/1000
    self.energy = None
    self.mass = mass
    self.cost = cost
    self.name = name
    self.dimensions = dimensions
    if self.dimensions['length'] is not None and self.dimensions['width'] is not None and self.dimensions['height'] is not None:
      self.volume = self.dimensions['length']*self.dimensions['width']*self.dimensions['height']
    else:
      self.volume = None
    self.power = None
    self.current = None
    self.resistance = None
    self.efficiency = None
    
    def __repr__(self):
      return f'{self.name} ({round(self.voltage,2)}V, {round(self.capacity,2)}Ah, {round(self.mass,2)} lbs'

class Battery():
  """ Defines a battery as a collection of cells. Cells are assumed to 
  be connected in series."""
  def __init__(self, cell=Cell(),
               motor_voltage=144, # V DC, nominal,
               n_cells=None, # number of cells in series
               ):
    self.cell = cell
    self.name = cell.name + ' Battery'
    if not n_cells:
      self.n_cells = cells_for_voltage(motor_voltage=motor_voltage, cell_voltage=cell.voltage)
    else:
      self.n_cells = n_cells
    self.voltage = self.n_cells*cell.voltage
    self.kWh = self.n_cells*cell.kWh
    self.energy = None
    if self.cell.mass is not None:
      self.mass = self.n_cells*cell.mass
    else:
      self.mass = None
    if cell.cost is not None:
      self.cost = self.n_cells*cell.cost
    else:
      self.cost = None
    if self.cell.volume is not None:
      self.volume = self.n_cells*self.cell.volume
    else:
      self.volume = None
    self.power = None
    self.current = None
    self.resistance = None
    self.efficiency = None
    
  def __repr__(self):
    return f'{self.name} ({round(self.voltage,2)}V, {round(self.kWh,2)}kWh, {round(self.mass,2)} lbs)'

def cells_for_voltage(motor_voltage=None, # motor V DC, nominal
                  cell_voltage=3.2, # individual cell V DC, nominal
                )->int: # number of cells required
    """Returns the number of cells required to achieve the desired motor voltage.
    Rounds up to the nearest integer.
    """
    return int(np.floor(motor_voltage/cell_voltage))


Motor voltage can vary. The NetGain Hyper9 HV motor is 144V nominal, but can increase up to 180V. The Tesla Model S motor is 375V nominal, but can increase up to 450V.

In [5]:
test_eq(cells_for_voltage(motor_voltage=144, cell_voltage=3.2), 45)
test_eq(cells_for_voltage(motor_voltage=144, cell_voltage=3.6), 40)
test_eq(cells_for_voltage(motor_voltage=180, cell_voltage=3.6), 50)

An initial target is 50 cells at 3.6V for a total voltage of 180V. This is a common voltage for electric vehicles. 

In [6]:
cell_voltage = 3.6 # V DC, nominal
motor_voltage = 180 # V DC, nominal
n_cells = cells_for_voltage(motor_voltage=motor_voltage, cell_voltage=cell_voltage)
print(f"Cells required for {motor_voltage} total V at {cell_voltage} V per cell: {n_cells}")

Cells required for 180 total V at 3.6 V per cell: 50


In [7]:
#| export
#| hide 
def miles_per_cell(cell=None, # cell object
                miles_per_kwh=5, # km per kWh
                )->float: # km per cell
    """Returns the number of km per cell.
    """
    if cell:
        return cell.kWh*miles_per_kwh
    else:
        raise ValueError("No cell object provided.")

@patch
def range(self:Battery,
        miles_per_kwh=5, # miles per kWh
        )->float: # miles range
    """Returns the range estimate for the battery.
    """
    return round(self.kWh*miles_per_kwh,2)

@patch(as_prop=True)
def cost_per_mile(self:Battery):
    """Returns the cost per mile for the battery.
    """
    return round(self.cost/self.range(miles_per_kwh=3),2)

## Range

Range depends on the miles per kWh obtained during driving. Estimates for miles per kWh range from 3 to 5, depending on driving conditions. We will use 3 miles per kWh as a conservative estimate.

In [8]:
miles_per_kwh = 3.5 # miles per kWh

In [9]:
cell = Cell(voltage=3.6, capacity=140)
test_eq(miles_per_cell(cell=cell, miles_per_kwh=miles_per_kwh), 1.764)

In [10]:
cell = Cell(voltage=3.6, capacity=140)
battery = Battery(cell=cell, motor_voltage=180)
test_eq(battery.range(miles_per_kwh=miles_per_kwh), 88.2)

## Sample Batteries

Let's look at the specs for different batteries.


In [11]:
batteries = []

#### 163Ah Lithium Battery L173F163B CALB LiFePO4 Prismatic Cell

[Link](https://www.electriccarpartscompany.com/163ah-calb-batteries-l173f163b)


- 163Ah, 3.2V
- L173F163B
- CALB Lithium LiFePO4
- Prismatic Cell Batteries
- 6.9L * 1.43W * 9.1H in
- 174.4 * 36.4 * 230.5 mm
- 6.68 Lbs. / 3.03 Kg


In [12]:
cell = Cell(name='CALB_163Ah',voltage=3.6,capacity=163,
                  dimensions={'length': 6.9, 'width': 1.43, 'height': 9.1},
                  mass=6.68,
                  cost=167.00)

battery = Battery(cell=cell, motor_voltage=180)
test_eq(battery.range(miles_per_kwh=miles_per_kwh), 102.69)
test_eq(battery.mass, 334.0)
test_eq(round(battery.kWh,2), 29.34)
test_eq(round(battery.cost,2), 8350.0)
print(battery)
batteries.append(battery)

CALB_163Ah Battery (180.0V, 29.34kWh, 334.0 lbs)


Here is a version using 48 cells, which may be convenient for BMS integration:

In [13]:
cell = Cell(name='CALB_163Ah',voltage=3.6,capacity=163,
                  dimensions={'length': 6.9, 'width': 1.43, 'height': 9.1},
                  mass=6.68,
                  cost=167.00)
battery = Battery(cell=cell, n_cells=48)
test_eq(battery.range(miles_per_kwh=miles_per_kwh), 98.58)
test_eq(battery.mass, 320.64)
test_eq(round(battery.kWh,2), 28.17)
test_eq(round(battery.cost,2), 8016.0)
print(battery)
batteries.append(battery)

CALB_163Ah Battery (172.8V, 28.17kWh, 320.64 lbs)


And here is the smallest possible battery pack that can generate 144V:

In [14]:
cell = Cell(name='CALB_163Ah',voltage=3.6,capacity=163,
                  dimensions={'length': 6.9, 'width': 1.43, 'height': 9.1},
                  mass=6.68,
                  cost=167.00)
battery = Battery(cell=cell, motor_voltage=144)
test_eq(battery.range(miles_per_kwh=miles_per_kwh), 82.15)
test_eq(battery.mass, 267.2)
test_eq(round(battery.kWh,2), 23.47)
test_eq(round(battery.cost,2), 6680.0)
print(battery)
batteries.append(battery)

CALB_163Ah Battery (144.0V, 23.47kWh, 267.2 lbs)


#### CA180FI 180Ah CALB LiFePO4 Lithium Battery

[Link](https://www.electriccarpartscompany.com/180Ah-CALB-UL-Certified-Batteries)


- 180Ah, 3.2V, 2C
- CA180FI
- CALB Lithium LiFePO4
- UL Certified!
- Prismatic Cell Batteries
- USA or China Stock
- 7.1L * 2.8W * 11H in
- 180 * 71 * 280 mm
- 12.6 Lbs. / 5.7 Kg

In [15]:
cell = Cell(name='CALB_180Ah', voltage=3.6,capacity=180,
                dimensions={'length': 7.1, 'width': 2.8, 'height': 11},
                mass=12.6,
                cost=171.00)
battery = Battery(cell=cell, motor_voltage=180)
test_eq(battery.range(miles_per_kwh=miles_per_kwh), 113.4)
test_eq(battery.mass, 630.0)
test_eq(round(battery.kWh,2), 32.4)
test_eq(round(battery.cost,2), 8550.0)
print(battery)
batteries.append(battery)

CALB_180Ah Battery (180.0V, 32.4kWh, 630.0 lbs)


#### ElectricGT OXDE 2.2kWh Battery Module

This is a battery module developed by ElectricGT
[Link](https://electricgt.com/ox-drive-batteries/)

The module has the following specs:

- 14.8V nominal, 2.2kWh, 151Ah
- Dimensions: 14.02" x 5.98" x 4.3"
- Weight: 25 lbs
- $695 each

In [16]:
cell = Cell(name='ElectricGT_OXDE',voltage=14.8,capacity=151,mass=25,cost=695)
battery = Battery(cell=cell, motor_voltage=180)
test_eq(battery.range(miles_per_kwh=miles_per_kwh), 93.86)
test_eq(battery.mass, 300)
test_eq(round(battery.kWh,2), 26.82)
test_eq(round(battery.cost,2), 8340)
print(battery)
batteries.append(battery)

ElectricGT_OXDE Battery (177.6V, 26.82kWh, 300 lbs)


#### Compare batteries in terms of range, total cost, and cost per km

We can compare the batteries in terms of range, weight, total cost, and cost per km.

In [17]:
df = pd.DataFrame([{'cell': battery.cell.name,
  'n_cells': battery.n_cells,
  'range': battery.range(miles_per_kwh=miles_per_kwh),
  'mass': battery.mass,
  'kWh': round(battery.kWh,2),
  'cost': round(battery.cost,2),
  'cost_per_mile': battery.cost_per_mile,
  'voltage': battery.voltage,
    }  for battery in batteries])
df.sort_values(by='cost_per_mile', ascending=True)

,cell,n_cells,range,mass,kWh,cost,cost_per_mile,voltage
3,CALB_180Ah,50,113.40,630.00,32.40,8550.0,87.96,180.0
0,CALB_163Ah,50,102.69,334.00,29.34,8350.0,94.86,180.0
1,CALB_163Ah,48,98.58,320.64,28.17,8016.0,94.86,172.8
2,CALB_163Ah,40,82.15,267.20,23.47,6680.0,94.86,144.0
4,ElectricGT_OXDE,12,93.86,300.00,26.82,8340.0,103.67,177.6


In [19]:
#| hide
import nbdev; nbdev.nbdev_export()